In [7]:
import preprocess

## 1. 표제어 사전 만들기 

In [39]:
execfile("preprocess.py")
lemm_parser=ParseLemmJson(r'법률 지식베이스\법령용어\법령용어(Json)\test_vocab.json')
lemm_list,uris=lemm_parser.open()
lemm_pd=lemm_parser.toPandas()
lemm_pd.head(1)

json file opened
The term of json object: 납세대상자 dict_keys(['http://www.aihub.or.kr/kb/law/name', 'http://www.aihub.or.kr/kb/law/related', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://www.w3.org/2000/01/rdf-schema#label'])
The term of json object: 촬영후 dict_keys(['http://www.aihub.or.kr/kb/law/name', 'http://www.aihub.or.kr/kb/law/related', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://www.w3.org/2000/01/rdf-schema#label'])
json file opened
The term of json object: 납세대상자 dict_keys(['http://www.aihub.or.kr/kb/law/name', 'http://www.aihub.or.kr/kb/law/related', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://www.w3.org/2000/01/rdf-schema#label'])
The term of json object: 촬영후 dict_keys(['http://www.aihub.or.kr/kb/law/name', 'http://www.aihub.or.kr/kb/law/related', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://www.w3.org/2000/01/rdf-schema#label'])


,vocab
0,납세대상자


## 2. 토큰화, 위치 라벨링 진행하기

### (1) Connect with API

In [6]:

execfile("preprocess.py")
dataParser=ParsePanrye()
soup=dataParser.connect2()


              class is initialized, 
              be sure your ip is authenticated by the api
              your ip: 192.168.56.1


In [7]:
soup.get_text()

'prec*evtNm829281221731강도·폭행·업무방해·부착명령2021도171312022.05.19대법원형사400102전원합의체 판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221731&type=HTML&mobileYn=221733채무불이행자명부등재2021마63712022.05.17대법원민사400101결정자/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221733&type=HTML&mobileYn=221741회계장부와서류의열람및등사청구의소2019다2701632022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221741&type=HTML&mobileYn=221739재산세부과처분취소2018두501472022.05.13대법원세무400108판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221739&type=HTML&mobileYn=221737무고·사문서위조·위조사문서행사2017도38842022.05.13대법원형사400102판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221737&type=HTML&mobileYn=221743물품대금2018다2247812022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221743&type=HTML&mobileYn=221735명예훼손2020도156422022.05.13대법원형사400102판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221735&type=HTML&mobileYn=221747손해배상(기)2019다2157912022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=p

### (2) 원하는 데이터만 추출해보기 

In [12]:
prec=[t.get_text() for t in soup.find_all("prec")]
for p in prec:
    print(p,"\n")

221731강도·폭행·업무방해·부착명령2021도171312022.05.19대법원형사400102전원합의체 판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221731&type=HTML&mobileYn= 

221733채무불이행자명부등재2021마63712022.05.17대법원민사400101결정자/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221733&type=HTML&mobileYn= 

221741회계장부와서류의열람및등사청구의소2019다2701632022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221741&type=HTML&mobileYn= 

221739재산세부과처분취소2018두501472022.05.13대법원세무400108판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221739&type=HTML&mobileYn= 

221737무고·사문서위조·위조사문서행사2017도38842022.05.13대법원형사400102판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221737&type=HTML&mobileYn= 

221743물품대금2018다2247812022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221743&type=HTML&mobileYn= 

221735명예훼손2020도156422022.05.13대법원형사400102판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221735&type=HTML&mobileYn= 

221747손해배상(기)2019다2157912022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&targ

In [14]:
print(type(prec[0]))

<class 'str'>


### 중간 체크업 

*상황* 
현재 target=prec으로 접근할 수 있는 판례 개수는 20개에 지나지 않으며, 아래와 같은 키의 구성으로 되어있다.

- 판례일련번호
- 사건명 (코드명일 때도 있고, ![[CDATA]]형일 때도 있음)
- 사건번호
- 선고일자 (사건번호와 선고일자는 (사건번호,신고일자) 튜플으로 판례/튜플 형식으로 판례일련번호를 대체할 수 있다.)
- 법원명
- 법원종류코드
- 사건종류명
- 사건종류코드
- 판결유형
- 선고
- 판례상세링크 

판례상세링크, 혹은 law.go.kr/판례/(사건번호,선고일자)로 판례 상세를 검색할 수 있다. 

판례 상세페이지는 제목과 요약, 본문으로 구성되어있는데 본문에는 수많은 다른 판례가 참조되어있다. 


*접근방법*<br>

#### 과제 1: 데이터 정제하기 
모든 판례에 대해서 db를 생성하는 과정으로 offline이다. 최대한 많은 판례를 처리하는 것이 목적이다.
처리 이후에는 수집한 데이터를 가지고 클러스터링 내지는 동질 그룹을 파악해 링크 또는 추천을 하는 것이다. 

- 판례가 api로 되는 것만 일단 부른다 (20개) 
- 먼저 판례 요약, 판례 본문, 제목을 literal로 불러온다.
- 판례 링크가 첨부되어있는 줄은 중요한 줄로 따로 위치와 링크를 저장해둔다. 
- 판례는 전부 [PANRYE]로 토큰화하고, 나머지는 일반적으로 토큰화한다.
- 표제어 사전을 거쳐 불필요한 토큰 분절을 방지한다. 

#### 과제 2: 답안지가 필요 없는 textRank로 중요 단어, 중요 문장 추출 시도하기
- TextRank를 연결한다

#### 과제 3: 판례문이 붙어있는 문장만 모아놓기
#### 과제 4: 추천시스템의 답을 찾기
가장 이상적인 것은 bert나 gpt-3로 중요 단어, 문장의 순위를 구하는 것인데 우리가 답안지가 없다. 

지금 판례들이 서로 링크가 되어있는데<br>
X판례 내용-->Y스스로가 링크된 판례를 가리키는 one hot vector 느낌의 답안지<br>
를 구성해서 딥러닝 모델의 판례 embedding을 구하고 나면<br>
클러스터링이 가능할 것으로 보인다. <br>

아니면 
X판례 내용 --> Y textRank 결과 단어들으로<br>
딥러닝 모델에 넣어 돌려 판례 embedding을 구하고 나면<br>
그걸로 클러스터링을 해도 괜찮을 것 같다.<br>

클러스터링의 유효성 판단은... 사용자 확인 외에 뭐가 또 있을까? 

### (3) literal 하게 본문 긁어오기 

In [33]:
execfile("preprocess.py")
dataParser=ParsePanrye()
dataParser.parsePrecDetail()


              class is initialized, 
              be sure your ip is authenticated by the api
              your ip: 192.168.56.1




















<html>
<head>
<script type="text/javascript" src="/DRF/js/jquery.js"></script>

	<style>
	body {
	    overflow: hidden; 
	    width: 100%;
		height : 100%;
	}
	
	#bodyDiv {
		width: 100%;
		height: 100%;
		overflow-x : hidden;
		overflow-y : auto;
	}
	
	</style>
</head>
<script type="text/javascript">
	$(document).ready(function(){
		// 모바일일 경우 수행		
		if($("#mobileYn").val() == 'Y'){
			var url = "mobilePrecInfoR.do";
			var param = "precSeq="+$("#precSeq").val(); 
			
			$.ajax({
				url: url,
				type:'POST',
				dataType:'text',
				data: param,
				timeout : 150000,
		        error : function(request,status,error){
		        	//alert("다시 실행해주시기 바랍니다. :: " + status);
		        	location.href = $("#url").val();
		        },
				success: function(responseData, result){
					if(responseData != ''){
						$("#bodyDiv").html(responseData)

TypeError: 'NoneType' object is not callable

## 3. 빈칸-답안 쌍 오브젝트 만들기 